In [108]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [109]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"


In [110]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [111]:
def file_paths(df):
    print(df['variable'])
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df[['path_2', 'path_3', 'path_4']] =  df[['path_2', 'path_3', 'path_4']].map(lambda x: np.nan if isinstance(x, str) and mark in x else x)


    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [112]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [113]:
def column_paths(df, row_var, get_var, unlist_var, explode_var):


    
    # Convert to dict if it's a string
    df[row_var] = df[row_var].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    
    # Extract nested values using keys
    df[unlist_var] = df.apply(lambda row: row[row_var].get(row[get_var], None) if isinstance (row[row_var], dict) else None, axis=1)


    # Assign list if the data in the original structure is a list
    
    def detect_list(x):
        if isinstance(x, list):
            return 'LIST'
        elif pd.isna(x):
            return 'MISSING'
        elif x == 'No data':
                return 'MISSING'
        else:
            return 'NO LIST'

    df[f'{explode_var}_LIST'] = df[unlist_var].apply(detect_list)

    #Unlist data originally listed to continue wit the extractions
    df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

    
    df[explode_var] = df[unlist_var]
    df = df.explode(explode_var)



    return df

In [114]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = '' 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    df['row_path'] = df['row_path'].astype('str').apply(eval)

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')



    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['row_path', 'col_path_1', 'col_path_2', 'col_path_3', 'col_path_4']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        row_var = unlist_var

    return df


In [115]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
                  'path_3', 'path_4', 'json_name', 'row_path', 
                  'col_path_1', 'col_path_1_LIST',
                  'col_path_2', 'col_path_2_LIST',
                  'col_path_3', 'col_path_3_LIST',
                  'col_path_4', 'col_path_4_LIST',
                  'col_path_5', 'col_path_5_LIST',
                  'data_type', 
                  'col_path_1_values','col_path_2_values',
                  'col_path_3_values', 'col_path_4_values', 
                  'col_path_5_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}Instagram/Output/Output_" + file_name + '.csv', index=False)

    return df

In [116]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    
    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)
    

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)
    
    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [117]:
input_directory = Path(f'{main_path}Instagram/Input_test')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/Instagram/Input_test


In [118]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file.name)
        structure_donations(file)
        

08-04-2025-json_structure instagram.json
0     followers_and_following/follow_requests_you've...
1              followers_and_following/followers_1.json
2                followers_and_following/following.json
3      followers_and_following/removed_suggestions.json
4     login_and_profile_creation/last_known_location...
5        login_and_profile_creation/login_activity.json
6     login_and_profile_creation/password_change_act...
7     login_and_profile_creation/profile_privacy_cha...
8        login_and_profile_creation/signup_details.json
9            information_about_you/possible_emails.json
10          information_about_you/profile_based_in.json
11     information_about_you/locations_of_interest.json
12    personal_information/instagram_profile_informa...
13       personal_information/personal_information.json
14    personal_information/professional_information....
15    autofill_information/in-app_browser_autofill_s...
16                      device_information/devices.json
17     

In [ ]:
# Path to the folder containing CSV files
output_path = f"{main_path}Instagram/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


# Concatenate all dataframes
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

df_final = merged_df
df_final = df_final.replace('Missing', np.nan)


# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Instagram/Final/Merged_structures_IG.csv", index=False)

/tmp/ipykernel_1791128/2936249484.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)
